#### Genism Word2Vec

In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

import re  # For preprocessing

from time import time  # To time our operations
from collections import defaultdict  # For word frequency

import spacy  # For preprocessing

from gensim.models import Word2Vec

# import logging  # Setting up the loggings to monitor gensim
# logging.basicConfig(format="%(levelname)s - %(asctime)s: %(message)s", datefmt= '%H:%M:%S', level=logging.INFO)

from sklearn.manifold import TSNE

from numpy import dot
from numpy.linalg import norm

D:\Anaconda-16-FEB\lib\site-packages\scipy\__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


## Dataset Description
The dataset used is from Kaggle Dataset( https://www.kaggle.com/CooperUnion/cardataset) 

This cars dataset includes features such as `make`, `model`, `year`, `engine`, and other properties of the car. 

We will use these features to generate the word embeddings for each make model and then compare the similarities between different make model. 

The following dataframe shows the detail information of this dataset.

In [2]:
location = 'https://github.com/gridflowai/gridflowAI-datasets-icons/raw/master/AI-DATASETS/01-MISC/word2vec-data.csv'

In [3]:
df = pd.read_csv(location)
df.sample(10)

,Make,Model,Year,Engine Fuel Type,Engine HP,Engine Cylinders,Transmission Type,Driven_Wheels,Number of Doors,Market Category,Vehicle Size,Vehicle Style,highway MPG,city mpg,Popularity,MSRP
6195,Dodge,Journey,2015,regular unleaded,283.0,6.0,AUTOMATIC,front wheel drive,4.0,Crossover,Midsize,4dr SUV,25,17,1851,31395
3518,Dodge,Daytona,1993,regular unleaded,141.0,6.0,MANUAL,front wheel drive,2.0,Hatchback,Compact,2dr Hatchback,25,17,1851,2000
8482,Suzuki,Reno,2008,regular unleaded,127.0,4.0,MANUAL,front wheel drive,4.0,Hatchback,Compact,4dr Hatchback,28,20,481,13839
2862,Bentley,Continental GT,2015,flex-fuel (premium unleaded required/E85),626.0,12.0,AUTOMATIC,all wheel drive,2.0,"Exotic,Flex Fuel,Factory Tuner,Luxury,High-Per...",Midsize,Convertible,20,12,520,250100
3522,Aston Martin,DB7,2003,premium unleaded (required),435.0,12.0,MANUAL,rear wheel drive,2.0,"Exotic,Factory Tuner,High-Performance",Compact,Coupe,15,9,259,156300
1105,Audi,A6,2015,diesel,240.0,6.0,AUTOMATIC,all wheel drive,4.0,"Diesel,Luxury",Midsize,Sedan,38,24,3105,62500
4162,Cadillac,Escalade,2015,flex-fuel (unleaded/E85),420.0,8.0,AUTOMATIC,four wheel drive,4.0,"Flex Fuel,Luxury",Large,4dr SUV,21,14,1624,74295
2082,Mercedes-Benz,C-Class,2017,premium unleaded (required),362.0,6.0,AUTOMATIC,all wheel drive,4.0,"Factory Tuner,Luxury,High-Performance",Midsize,Sedan,29,21,617,52000
10402,Hyundai,Tiburon,2008,regular unleaded,172.0,6.0,MANUAL,front wheel drive,2.0,Hatchback,Compact,2dr Hatchback,24,17,1439,20170
4661,Ford,Festiva,1991,regular unleaded,63.0,4.0,MANUAL,front wheel drive,2.0,Hatchback,Compact,2dr Hatchback,38,30,5657,2000


In [4]:
df.shape

(11914, 16)

#### Genism word2Vec 

requires that a format of `list of list` for training where every __document is contained in a list and every list contains list of tokens of that document__. 

At first, we need to generate a format of `list of list` for training the make model word embedding. 

To achieve these, we need to do the following data preprocessing steps :

- Create a new column for __Make Model__
- Generate a format of list of list for each __Make Model__ with the following features: 
    - Engine Fuel Type, 
    - Transmission Type, 
    - Driven_Wheels, 
    - Market Category, 
    - Vehicle Size and 
    - Vehicle Style.

Create a new column for Make Model

In [5]:
df['Maker_Model'] = df['Make']+ " " + df['Model']

Generate a format of list of list for each Make Model

In [6]:
df1 = df[['Engine Fuel Type',
          'Transmission Type',
          'Driven_Wheels',
          'Market Category',
          'Vehicle Size', 
          'Vehicle Style', 
          'Maker_Model']]

df1.head()

,Engine Fuel Type,Transmission Type,Driven_Wheels,Market Category,Vehicle Size,Vehicle Style,Maker_Model
0,premium unleaded (required),MANUAL,rear wheel drive,"Factory Tuner,Luxury,High-Performance",Compact,Coupe,BMW 1 Series M
1,premium unleaded (required),MANUAL,rear wheel drive,"Luxury,Performance",Compact,Convertible,BMW 1 Series
2,premium unleaded (required),MANUAL,rear wheel drive,"Luxury,High-Performance",Compact,Coupe,BMW 1 Series
3,premium unleaded (required),MANUAL,rear wheel drive,"Luxury,Performance",Compact,Coupe,BMW 1 Series
4,premium unleaded (required),MANUAL,rear wheel drive,Luxury,Compact,Convertible,BMW 1 Series


In [7]:
pd.set_option('max_colwidth', 140)

In [8]:
# For each row, combine all the columns into one column
df2 = df1.apply(lambda x: ','.join(x.astype(str)), axis=1) 

df2.head()

0    premium unleaded (required),MANUAL,rear wheel drive,Factory Tuner,Luxury,High-Performance,Compact,Coupe,BMW 1 Series M
1                   premium unleaded (required),MANUAL,rear wheel drive,Luxury,Performance,Compact,Convertible,BMW 1 Series
2                    premium unleaded (required),MANUAL,rear wheel drive,Luxury,High-Performance,Compact,Coupe,BMW 1 Series
3                         premium unleaded (required),MANUAL,rear wheel drive,Luxury,Performance,Compact,Coupe,BMW 1 Series
4                               premium unleaded (required),MANUAL,rear wheel drive,Luxury,Compact,Convertible,BMW 1 Series
dtype: object

In [9]:
# Store them in the pandas dataframe
df_clean = pd.DataFrame({'clean': df2}) 
df_clean.head()

,clean
0,"premium unleaded (required),MANUAL,rear wheel drive,Factory Tuner,Luxury,High-Performance,Compact,Coupe,BMW 1 Series M"
1,"premium unleaded (required),MANUAL,rear wheel drive,Luxury,Performance,Compact,Convertible,BMW 1 Series"
2,"premium unleaded (required),MANUAL,rear wheel drive,Luxury,High-Performance,Compact,Coupe,BMW 1 Series"
3,"premium unleaded (required),MANUAL,rear wheel drive,Luxury,Performance,Compact,Coupe,BMW 1 Series"
4,"premium unleaded (required),MANUAL,rear wheel drive,Luxury,Compact,Convertible,BMW 1 Series"


In [10]:
# Create the list of list format of the custom corpus for gensim modeling 
sent = [row.split(',') for row in df_clean['clean']]

In [11]:
len(sent)

11914

In [12]:
# show the example of list of list format of the custom corpus for gensim modeling 
sent[:2]

[['premium unleaded (required)',
  'MANUAL',
  'rear wheel drive',
  'Factory Tuner',
  'Luxury',
  'High-Performance',
  'Compact',
  'Coupe',
  'BMW 1 Series M'],
 ['premium unleaded (required)',
  'MANUAL',
  'rear wheel drive',
  'Luxury',
  'Performance',
  'Compact',
  'Convertible',
  'BMW 1 Series']]

#### Genism word2vec Model Training

- __size__: The number of dimensions of the embeddings and the default is 100.
- __window__: The maximum distance between a target word and words around the target word. The default window is 5.
- __min_count__: The minimum count of words to consider when training the model; words with occurrence less than this count will be ignored. The default for min_count is 5.
- __workers__: The number of partitions during training and the default workers is 3.

- __sg__: The training algorithm, either CBOW(0) or skip gram (1). 

The default training alogrithm is CBOW.

In [13]:
## Train the genisim word2vec model with our own custom corpus
model = Word2Vec(sent, 
                 min_count = 2,
                 vector_size=50,
                 workers   = 3, 
                 window    = 3, 
                 sg        = 1)

In [14]:
# Get the vocabulary
vocab = model.wv.index_to_key

# Print first 10 words as an example
print(vocab[:10])

['AUTOMATIC', 'regular unleaded', 'front wheel drive', 'Compact', 'Midsize', 'nan', 'rear wheel drive', 'Luxury', 'Sedan', 'MANUAL']


In [15]:
len(vocab)

936

In [21]:
## We can obtain the word embedding directly from the training model
model.wv['manual']

KeyError: "Key 'manual' not present"

#### Compare Similarities

use Word2vec to compute similarity between two make model in the vocabulary by invoking the model.similarity() and passing in the relvevant words. 

For instance, model.similarity(`Porsche 718 Cayman`, `Nissan Van`) 

This will give us the `Euclidian similarity` between `Porsche 718 Cayman` and `Nissan Van`.

In [ ]:
model.wv.similarity('Porsche 718 Cayman', 'Toyota Tercel')

In [16]:
model.wv.similarity('Porsche 718 Cayman', 'Toyota Tercel')

0.8267118

From the above example, we can tell that Porsche 718 Cayman is more similar with Mercedes-Benz SLK-Class than Nissan Van. 



In [17]:
model.wv.similarity('Hyundai Accent', 'Mercedes-Benz SLK-Class')

0.7540901

#### __model.most_similar()__ 

In [18]:
## Show the most similar vehicles for Mercedes-Benz SLK-Class : Default by eculidean distance 
model.wv.most_similar('Mercedes-Benz SLK-Class')[:5]

[('Ferrari California', 0.9897400140762329),
 ('Lotus Elise', 0.9895458817481995),
 ('Mercedes-Benz SL-Class', 0.9886258840560913),
 ('Ferrari 458 Italia', 0.9881365299224854),
 ('Mercedes-Benz CLK-Class', 0.9878443479537964)]

In [20]:
model.wv.most_similar('MANUAL')[:15]

[('Pontiac G5', 0.762554943561554),
 ('Nissan 300ZX', 0.7583836913108826),
 ('Volkswagen Cabrio', 0.7482433319091797),
 ('UNKNOWN', 0.746385931968689),
 ('Chrysler Prowler', 0.7436459064483643),
 ('Volkswagen New Beetle', 0.7357469797134399),
 ('Chrysler Crossfire', 0.7343031764030457),
 ('Dodge Stealth', 0.7331790924072266),
 ('Chrysler Le Baron', 0.7297907471656799),
 ('Mazda MX-5 Miata', 0.7293539047241211)]

In [26]:
## Show the most similar vehicles for Toyota Camry : Default by eculidean distance 
model.wv.most_similar('Toyota Camry')[:15]

[('Hyundai Sonata', 0.9861137270927429),
 ('Toyota Avalon', 0.9856813549995422),
 ('Nissan Altima', 0.9830912947654724),
 ('Nissan Sentra', 0.9830172657966614),
 ('Buick Verano', 0.9784183502197266),
 ('Chevrolet Cruze', 0.976456344127655),
 ('Volkswagen Passat', 0.9762012958526611),
 ('Ford Five Hundred', 0.9731436967849731),
 ('Chevrolet Malibu', 0.9727350473403931),
 ('Oldsmobile Alero', 0.9715654253959656)]

However, `Euclidian similarity` cannot work well for the `high-dimensional word vectors`, 

This is because Euclidian similarity will increase the number of dimensions increases even if the word embedding stands for different meanings. 

Alternatively, we can use `cosine similarity` to measure the similarity between two vectors. 

Under cosine similarity, no similarity is expressed as a 90-degree angle while the total similarity of 1 is at 0 degree angle.

In [27]:
def cosine_sim (model, word, target_list , num) :
    cosine_dict = {}
    word_list   = []
    a           = model[word]
    
    for item in target_list :
        if item != word :
            b = model [item]
            
            cos_sim = dot(a, b)/(norm(a)*norm(b))
            
            cosine_dict[item] = cos_sim
            
    dist_sort=sorted(cosine_dict.items(), key=lambda dist: dist[1],reverse = True) ## in Descedning order 
    
    for item in dist_sort:
        word_list.append((item[0], item[1]))
        
    return word_list[0:num]

In [28]:
list(df1.Maker_Model.unique())[:15]

['BMW 1 Series M',
 'BMW 1 Series',
 'Audi 100',
 'FIAT 124 Spider',
 'Mercedes-Benz 190-Class',
 'BMW 2 Series',
 'Audi 200',
 'Chrysler 200',
 'Nissan 200SX',
 'Nissan 240SX',
 'Volvo 240',
 'Mazda 2',
 'BMW 3 Series Gran Turismo',
 'BMW 3 Series',
 'Mercedes-Benz 300-Class']

In [37]:
Maker_Model = list(df1.Maker_Model.unique()) ## only get the unique Maker_Model_Year

## Show the most similar Mercedes-Benz SLK-Class by cosine similarity
cosine_sim (model, 'Mercedes-Benz SLK-Class', Maker_Model, 5)

[('BMW M6', 0.99418545),
 ('Nissan GT-R', 0.9940916),
 ('Aston Martin V12 Vanquish', 0.9935934),
 ('Audi S5', 0.9934372),
 ('Mercedes-Benz SLS AMG GT', 0.99321395)]